In [32]:
import pandas as pd 
import json 
import shapefile
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import os 
import json
import numpy as np

def isNested(entry): 
    try: 
        sum(entry)
        return 0 
    except: 
        return 1

def readInJson(fileName):
    try: 
        df = pd.read_json(fileName)
    except: 
        handle = json.load(open(fileName))
        df = pd.DataFrame(handle['features'])
        
    return df

In [35]:
garbageCollectionDate = sorted([i for i in os.listdir() if i.endswith('json')])

In [52]:
lonlatSuburb = pd.read_csv('lonlatSuburb.csv')
vicPostcodes = pd.read_csv('vicPostcodes.csv')

lonlatSuburb['lat'] = lonlatSuburb.lat.apply(lambda x: round(x, 5))
lonlatSuburb['long'] = lonlatSuburb.long.apply(lambda x: round(x, 5))

vicPostcodes['lat'] = vicPostcodes.lat.apply(lambda x: round(x, 6))
vicPostcodes['long'] = vicPostcodes.long.apply(lambda x: round(x, 6))

In [59]:
garbageCollectionDate[13]

'Southern-Grampians-garbage-collection-zones.json'

In [38]:
readInJson(garbageCollectionDate[12]).head(2)

,type,id,geometry,geometry_name,properties
0,Feature,ckan_58c5f1fa_5e68_41db_a765_beda49521eea.1,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",geom,"{'name': 'Tuesday Zone 4', 'rub_day': 'Tuesday..."
1,Feature,ckan_58c5f1fa_5e68_41db_a765_beda49521eea.2,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",geom,"{'name': 'Tuesday Zone 4', 'rub_day': 'Tuesday..."


In [53]:
df = readInJson(garbageCollectionDate[12])

df['geo'] = df.geometry.apply(lambda x: x['coordinates'][0][0])

df = df.explode('geo')

df['geo_nested'] = df.geo.apply(lambda x: isNested(x))
df['lat'] = df.geo.apply(lambda x: round(x[0], 5))
df['long'] = df.geo.apply(lambda x: round(x[1], 5))

df.drop(['geo','id', 'geometry', 'geometry_name',
         'geo_nested', 'type'], axis = 1, inplace = True)

In [54]:
def collectionFrequency(row, wasteType): 
    
    rub_colDay = row[f'{wasteType}_day']
    rub_colFreq = row[f'{wasteType}_weeks']
    
    if rub_colFreq == '2' or rub_colFreq == 2: 
        rub_colFreq = 'Every Second '
    else: 
        rub_colFreq = 'Every '
        
    if rub_colDay in [None, 'None']:
        rub_colDay = ''
    elif rub_colFreq == [None, 'None']: 
        rub_colFreq = ''

    return rub_colFreq + rub_colDay

df['landfill_colDay'] = df.properties.apply(lambda x: collectionFrequency(x, 'rub'))

df['landfill_colDay'] = df.properties.apply(lambda x: collectionFrequency(x, 'rub'))
df['recyclable_colDay'] = df.properties.apply(lambda x: collectionFrequency(x, 'rec'))
df['green_colDay'] = df.properties.apply(lambda x: collectionFrequency(x, 'grn'))

df.loc[df.green_colDay == 'Every ', 'green_colDay'] = 'Unavailable'

In [57]:
wangaratt = lonlatSuburb.merge(df)[['locality', 'long', 'lat', 'landfill_colDay', 'recyclable_colDay']]\
.sort_values(['locality', 'landfill_colDay', 'recyclable_colDay'])\
        .drop_duplicates(['locality','recyclable_colDay', 'landfill_colDay', 'recyclable_colDay'])\
        .sort_values(['locality'])\
.merge(vicPostcodes[['locality', 'postcode']])\
.sort_values(['postcode', 'landfill_colDay', 'recyclable_colDay'])\
.drop_duplicates(subset = ['postcode', 'landfill_colDay', 'recyclable_colDay'])\
.rename(columns = dict(zip(['landfill_colDay', 'landfill_colDay', 'recyclable_colDay'], 
                           ['landfill', 'landfill', 'recyclable'])))\
.drop(['long', 'lat', 'locality'], axis = 1)\
.melt(id_vars = 'postcode')\
.rename(columns = {'variable':'binInfo_bin_type', 'value':'collection_time'})\
.drop_duplicates(['postcode', 'binInfo_bin_type'])

wangaratt = wangaratt.loc[wangaratt.collection_time!= 'Unavailable']

In [58]:
wangaratt.to_csv('../GBD_wangaratta.csv', index = False)